In [1]:
from pyCode.AuxDecays import DecayList,Decay
from pyCode.component import Component
from pyCode.EqFunctions import T as Tfunc
from pyCode.EqFunctions import H as Hfunc

INFO:pyCode.AuxFuncs:Loading aux functions. Ignoring BSM corrections to g* and g*_S


In [2]:
import numpy as np
import sympy as sp

In [3]:
lamb = 0.17
decays = DecayList()
decayToDM = Decay(instate='Mediator',fstates=['DM','radiation'],br=1.)
decays.addDecay(decayToDM)
decays.Xfraction = 0.1
decays.width = 2.49e-15*(lamb/4.3e-7)**2
dofDM = -2
dofMed = 6
dm = Component(label='DM',Type='thermal',dof=dofDM, mass=500.)
mediator = Component(label='Mediator',Type='thermal',dof=dofMed, mass=510.,decays=decays)
components = [dm,mediator]

In [5]:
#Store the number of components:
nComp = len(components)

n = np.array(sp.symbols('n:%d'%nComp))
Ri = np.array(sp.symbols('R:%d'%nComp))
NS = sp.symbols('NS')
x = sp.symbols('x')
T = Tfunc(x,NS)

#Current energy densities:
rho = n*Ri

#Compute equilibrium densities:
neq = np.array([comp.nEQ(T) for comp in components])

#Compute ratio of equilibrium densities
#(helps with numerical instabilities)
#rNeq[i,j] = neq[i]/neq[j]
rNeq = np.array([[compi.rNeq(T,compj) if compi.active and compj.active else 0. for compj in components] 
                 for compi in components])

#Dictionary with label:index mapping:
labelsDict = dict([[comp.label,i] for i,comp in enumerate(components)])
isActive = np.array([comp.active for comp in components])

rhoTot = np.sum(rho,where=isActive,initial=0)

#Compute Hubble factor:
H = Hfunc(T,rhoTot)

#Effective equilibrium densities and BRs:
NXth = np.array([comp.getNXTh(T,n,rNeq,labelsDict) for comp in components])
widths = np.array([comp.width(T) for comp in components])
masses = np.array([comp.mass(T) for comp in components])
BRX = np.array([comp.getBRX(T) for comp in components])
sigmaV = np.array([comp.getSIGV(T) for comp in components])

In [6]:
RHS = -3*n
#Decay term:
RHS -= widths*masses*n/(Ri)
#Inverse decay term:
RHS += widths*masses*NXth/(Ri) #NXth should be finite if i -> j +..
#Annihilation term:            
RHS += sigmaV*(neq - n)*(neq + n)/H

In [15]:
RHS[1]

-3*n1 + 0.198486262844781*n0*((-Heaviside(0.1 - 0.002*T(x, NS), 1.0)*Heaviside(0.1 - 0.00196078431372549*T(x, NS), 1.0) + 1.0)*(24432.1407166871*pi**(-1.5)*(3.15383506343714e-6*T(x, NS)**2 + 0.00367647058823529*T(x, NS) + 1.0)*T(x, NS)**1.5*exp(-510.0/T(x, NS))*Heaviside(0.1 - 0.00196078431372549*T(x, NS), 1.0) + 7.21234141895757*T(x, NS)**3*Heaviside(0.00196078431372549*T(x, NS) - 1.5, 0.0)/pi**2 + 780300.0*T(x, NS)*Heaviside(1.5 - 0.00196078431372549*T(x, NS), 1.0)*Heaviside(0.00196078431372549*T(x, NS) - 0.1, 0.0)*besselk(2, 510.0/T(x, NS))/pi**2)/(7905.69415042095*pi**(-1.5)*(3.28125e-6*T(x, NS)**2 + 0.00375*T(x, NS) + 1.0)*T(x, NS)**1.5*exp(-500.0/T(x, NS))*Heaviside(0.1 - 0.002*T(x, NS), 1.0) + 1.80308535473939*T(x, NS)**3*Heaviside(0.002*T(x, NS) - 1.5, 0.0)/pi**2 + 250000.0*T(x, NS)*Heaviside(1.5 - 0.002*T(x, NS), 1.0)*Heaviside(0.002*T(x, NS) - 0.1, 0.0)*besselk(2, 500.0/T(x, NS))/pi**2) + 3.0904485111388*(3.15383506343714e-6*T(x, NS)**2 + 0.00367647058823529*T(x, NS) + 1.0)*e

In [8]:
fff = sp.lambdify([x,n,Ri,NS],RHS, modules=[{'DiracDelta' : lambda z: 0.},'numpy','sympy'])

In [9]:
fff(1.,np.array([10.,1.]),np.array([600.,600.]),2.)

[-30.0, nan]

In [10]:
ff = sp.lambdify([x,NS],Tfunc(x,NS))

In [11]:
ff(1.,2.)

0.25406166871224

In [9]:
vv = np.hstack((n,Ri,[T]))
jac = sp.Matrix(RHS).jacobian(vv)
jacf = sp.lambdify(vv,jac,modules=[{'DiracDelta' : lambda x: 0.},'numpy','sympy'])

ValueError: 
Can't calculate derivative wrt T(x, NS).

In [ ]:
jacf(10.,1.,600.,600.,1000.)